# Module 3: SQL (Database Schema Design, Data Ingestion, SQL Queries)

For the third module, the following tasks were performed:
#### 1. Database Schema Design:
- Design a relational database schema to store the telco customer churn data, including raw data and prediction results
- Define tables for customers, services, billing, churn predictions
#### 2. Data Ingestion
- Write SQL queries to create the database schema
- Import the preprocessed dataset into the SQL database
- Ensure data integrity and consistency with primary and foreign keys


In [1]:
import psycopg2
import pandas as pd

##### Formatting the Raw Dataset to be better suited for SQL

In [2]:
df = pd.read_csv('../data/Telco-Customer-Churn.csv')
true_false_map = {'Yes': True, 'No': False}
df2 = df.replace({'No phone service': 'No', 'No internet service': 'No'})
df_encoded = df2.replace(true_false_map)
df_encoded['SeniorCitizen'] = df_encoded['SeniorCitizen'].astype(bool)
df_encoded.to_csv('../data/sql_data.csv', index=False)
df_encoded

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,False,True,False,1,False,False,DSL,False,...,False,False,False,False,Month-to-month,True,Electronic check,29.85,29.85,False
1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,True,...,True,False,False,False,One year,False,Mailed check,56.95,1889.50,False
2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,True,...,False,False,False,False,Month-to-month,True,Mailed check,53.85,108.15,True
3,7795-CFOCW,Male,False,False,False,45,False,False,DSL,True,...,True,True,False,False,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,False,...,False,False,False,False,Month-to-month,True,Electronic check,70.70,151.65,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,6840-RESVB,Male,False,True,True,24,True,True,DSL,True,...,True,True,True,True,One year,True,Mailed check,84.80,1990.50,False
7028,2234-XADUH,Female,False,True,True,72,True,True,Fiber optic,False,...,True,False,True,True,One year,True,Credit card (automatic),103.20,7362.90,False
7029,4801-JZAZL,Female,False,True,True,11,False,False,DSL,True,...,False,False,False,False,Month-to-month,True,Electronic check,29.60,346.45,False
7030,8361-LTMKD,Male,True,True,False,4,True,True,Fiber optic,False,...,False,False,False,False,Month-to-month,True,Mailed check,74.40,306.60,True


##### Generating a Connection to a PostgreSQL Database

In [3]:
# conn = psycopg2.connect(
#     host='<Your Host - usually local host>',
#     dbname='<Your Database Name>',
#     user='<Your User - usually postgres>',
#     password='<Your Password>',
#     port=<Your Port - Usually 5432>
# )
conn = psycopg2.connect(
    host='localhost',
    dbname='CustomerChurnData',
    user='postgres',
    password='1234',
    port=5432
)
cursor = conn.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchall()
print(f"Connected to {record}")

Connected to [('PostgreSQL 14.12 (Ubuntu 14.12-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)]


### 1. Database Schema and Creating Tables in the database
Based on the data, we create the following tables for our schema:
- Customer - this table is basically the customer demographics, and stores gender, senior citizen, partner, dependents, and tenue
- Services - this table stores the services that the customer is availing such as phone service, internet service, tech support etc
- Billings - this table stores the customer billing information such as their contract, payment method, their monthly and total charges
- Churn Predictions - this table stores the customer churn data; the actua churn value and the predicted churn value through our machine learning model

![schema](schema.png)

In [4]:
cursor.execute("""
BEGIN;


CREATE TABLE IF NOT EXISTS public.customers (
    customer_id character varying(15) NOT NULL,
    gender character varying(7) NOT NULL,
    senior_citizen boolean NOT NULL,
    partner boolean NOT NULL,
    dependents boolean NOT NULL,
    tenure integer NOT NULL,
    PRIMARY KEY (customer_id)
);

CREATE TABLE IF NOT EXISTS public.services (
    customer_id character varying(15) NOT NULL,
    phone_service boolean NOT NULL,
    multiple_lines boolean NOT NULL,
    internet_service character varying(20) NOT NULL,
    online_security boolean NOT NULL,
    online_backup boolean NOT NULL,
    device_protection boolean NOT NULL,
    tech_support boolean NOT NULL,
    streaming_tv boolean NOT NULL,
    streaming_movies boolean NOT NULL
);

CREATE TABLE IF NOT EXISTS public.billing (
    customer_id character varying(15) NOT NULL,
    contract character varying(25) NOT NULL,
    paperless_billing boolean NOT NULL,
    payment_method character varying(50) NOT NULL,
    monthly_charges numeric NOT NULL,
    total_charges numeric NOT NULL
);

CREATE TABLE IF NOT EXISTS public.churn_predictions (
    customer_id character varying(15) NOT NULL,
    churn boolean NOT NULL,
    churn_prediction boolean NOT NULL
);

ALTER TABLE IF EXISTS public.services
    ADD CONSTRAINT customer_id FOREIGN KEY (customer_id)
    REFERENCES public.customers (customer_id) MATCH SIMPLE
    ON UPDATE CASCADE
    ON DELETE CASCADE
    NOT VALID;


ALTER TABLE IF EXISTS public.billing
    ADD CONSTRAINT customer_id FOREIGN KEY (customer_id)
    REFERENCES public.customers (customer_id) MATCH SIMPLE
    ON UPDATE CASCADE
    ON DELETE CASCADE
    NOT VALID;


ALTER TABLE IF EXISTS public.churn_predictions
    ADD CONSTRAINT customer_id FOREIGN KEY (customer_id)
    REFERENCES public.customers (customer_id) MATCH SIMPLE
    ON UPDATE CASCADE
    ON DELETE CASCADE
    NOT VALID;

END;
""")

conn.commit()

### 2. Importing the Preprocessed Data into the SQL database

The processed data for SQL from the Raw Data and the prediction results is now loaded, separated into different data frames with specific columns as per their corresponding tables, and then scripts are run to load the data into the connected SQL database.

In [5]:
# Read the data
data = pd.read_csv('../data/sql_data.csv')
churn_predictions = pd.read_csv('../data/prediction_results.csv')

customers_table = data[['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure']]
for index, row in customers_table.iterrows():
    cursor.execute("""
    INSERT INTO public.customers (customer_id, gender, senior_citizen, partner, dependents, tenure) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (customer_id) DO NOTHING""", 
    (row['customerID'], row['gender'], row['SeniorCitizen'], row['Partner'], row['Dependents'], row['tenure'])
    )

services_table = data[['customerID', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']]
for index, row in services_table.iterrows():
    cursor.execute("""
    INSERT INTO public.services (customer_id, phone_service, multiple_lines, internet_service, online_security, online_backup, device_protection, tech_support, streaming_tv, streaming_movies) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", 
    (row['customerID'], row['PhoneService'], row['MultipleLines'], row['InternetService'], row['OnlineSecurity'], row['OnlineBackup'], row['DeviceProtection'], row['TechSupport'], row['StreamingTV'], row['StreamingMovies'])
    )

billing_table = data[['customerID', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']]
for index, row in billing_table.iterrows():
    cursor.execute("""
    INSERT INTO public.billing (customer_id, contract, paperless_billing, payment_method, monthly_charges, total_charges) VALUES (%s, %s, %s, %s, %s, %s)""",
    (row['customerID'], row['Contract'], row['PaperlessBilling'], row['PaymentMethod'], row['MonthlyCharges'], row['TotalCharges'])
    )

churn_predictions_table = churn_predictions[['customerID', 'Actual', 'Predicted']]
churn_predictions_table = churn_predictions_table.replace({0: False, 1: True})
for index, row in churn_predictions_table.iterrows():
    cursor.execute("""
    INSERT INTO public.churn_predictions (customer_id, churn, churn_prediction) VALUES (%s, %s, %s)""",
    (row['customerID'], row['Actual'], row['Predicted'])
    )


conn.commit()

### 3. Querying the data

The loaded data is now queried based on various criteria such as tenure, monthly charges, contract types etc, to generate summary reports.

In [4]:
def show(query):
    cursor.execute(query)
    data = cursor.fetchall()
    for row in data:
        print(row)

#### - Querying on Tenure

- The **longest tenure** is of **72 months (6 years)** and **362 customers** have this tenure, while **only 1** was predicted to churn
- The **shortest tenure** is of only **1 month** and **613 customers** have this tenure, while **463** were predicted to churn
- **2058 customers** have had a tenure of **less than 1 year**, **117 customers** have had a tenure of exactly **1 year**, and **4857 customers** have had a tenure of **more than 1 year**
- From those who's tenure was **less than 1 year**, **1129** were predicted to churn
- The **average tenure** has been of about **32 months** (2 years and 8 months)

In [9]:
print("Longest Tenure:", end=" "); show(''' select max(tenure) from customers ''')
print("Customers with Longest Tenure:", end=" "); show(''' select count(*) from customers where tenure = (select max(tenure) from customers) ''')
print("Shortest Tenure:", end=" "); show(''' select min(tenure) from customers ''')
print("Customers with Shortest Tenure:", end=" "); show(''' select count(*) from customers where tenure = (select min(tenure) from customers) ''')
print("Customers with Tenure Less Than 1 Year:", end=" "); show(''' select count(*) from customers where tenure < 12 ''')
print("Customers with Tenure of 1 Year:", end=" "); show(''' select count(*) from customers where tenure = 12 ''')
print("Customers with Tenure More Than 1 Year:", end=" "); show(''' select count(*) from customers where tenure > 12 ''')
print("Average Tenure:", end=" "); show(''' select avg(tenure) from customers ''')
print("Average, Max, Min, and Standard Deviation of Tenure:", end=" "); show(''' select avg(tenure), max(tenure), min(tenure), stddev(tenure) from customers ''')
print("Number of Customers with Max Tenure predicted to churn"); show('''select count(*) as MaxTenurePredictedChurn from customers c
join churn_predictions cp on c.customer_id = cp.customer_id
where c.tenure = (select max(tenure) from customers) and cp.churn_prediction = true;''')
print("Number of Customers with Min Tenure predicted to churn", end=' '); show('''select count(*) as MinTenurePredictedChurn from customers c
join churn_predictions cp on c.customer_id = cp.customer_id
where c.tenure = (select min(tenure) from customers) and cp.churn_prediction = true;''')
print("Customers predicted to churn with less than 1 year", end=' '); show('''select count(*) as PredictedChurnLessThan1Year from customers c
join churn_predictions cp on c.customer_id = cp.customer_id
where c.tenure < 12 and cp.churn_prediction = true;''')

Longest Tenure: (72,)
Customers with Longest Tenure: (362,)
Shortest Tenure: (1,)
Customers with Shortest Tenure: (613,)
Customers with Tenure Less Than 1 Year: (2058,)
Customers with Tenure of 1 Year: (117,)
Customers with Tenure More Than 1 Year: (4857,)
Average Tenure: (Decimal('32.4217861205915813'),)
Average, Max, Min, and Standard Deviation of Tenure: (Decimal('32.4217861205915813'), 72, 1, Decimal('24.5452597092632553'))
Number of Customers with Max Tenure predicted to churn
(1,)
Number of Customers with Min Tenure predicted to churn (463,)
Customers predicted to churn with less than 1 year (1129,)


#### - Querying on Monthly Charges

- The **highest monthly charges** were **$118.75** of only one customer, who was availing all the services
- The **lowest monthly charges** were **$18.25** of only one customer, who was only availing one service (Phone Service)
- The **average monthly charges** were **$64.8** ($65 rounded up) 
- **3130** customers are billed less than or equal to the average charges, while the remaining **3902** are billed more than the average charges

In [10]:
print("Highest Monthly Charges:", end=" "); show(''' select max(monthly_charges) from billing ''')
# print("Customers with Highest Monthly Charges:", end=" "); show(''' select count(*) from billing where monthly_charges = (select max(monthly_charges) from billing) ''')
print("Total Services Used by highest monthly charges customer:", end=' '); show(''' select sum(
    (case when phone_service then 1 else 0 end) +
    (case when multiple_lines then 1 else 0 end) +
    (case when internet_service is not null then 1 else 0 end) +
    (case when online_security then 1 else 0 end) +
    (case when online_backup then 1 else 0 end) +
    (case when device_protection then 1 else 0 end) +
    (case when tech_support then 1 else 0 end) +
    (case when streaming_tv then 1 else 0 end) +
    (case when streaming_movies then 1 else 0 end)
) as total_services
from public.services
where customer_id in (
    select customer_id
    from public.billing
    where monthly_charges = (select max(monthly_charges) from public.billing)
); ''')

print("\nLowest Monthly Charges:", end=" "); show(''' select min(monthly_charges) from billing ''')
print("Services Used by lowest monthly charges customer:", end=' '); show(''' select sum(
    (case when phone_service then 1 else 0 end) +
    (case when multiple_lines then 1 else 0 end) +
    (case when online_security then 1 else 0 end) +
    (case when online_backup then 1 else 0 end) +
    (case when device_protection then 1 else 0 end) +
    (case when tech_support then 1 else 0 end) +
    (case when streaming_tv then 1 else 0 end) +
    (case when streaming_movies then 1 else 0 end)
) as total_services
from public.services
where customer_id in (
    select customer_id
    from public.billing
    where monthly_charges = (select min(monthly_charges) from public.billing)
); ''')

print("\nAverage Monthly Charges:", end=" "); show(''' select avg(monthly_charges) from billing ''')

print("\nCustomers with less than or equal to average, and more than average monthly charges:"); show(''' select
	case
		when monthly_charges < 65 then '<$65 (Average)'
		else '>$65 (More than Average)'
	end as mcr,
	count(*) as NumberOfCustomers
from billing
group by mcr
order by mcr; ''')

print("Monthly Charges statistics(Average, Max, Min, Standard Deviation):", end=' '); show(''' SELECT 
    AVG(monthly_charges) AS average_monthly_charges,
    MAX(monthly_charges) AS max_monthly_charges,
    MIN(monthly_charges) AS min_monthly_charges,
    STDDEV(monthly_charges) AS stddev_monthly_charges
FROM public.billing;
 ''')

Highest Monthly Charges: (Decimal('118.75'),)
Total Services Used by highest monthly charges customer: (9,)

Lowest Monthly Charges: (Decimal('18.25'),)
Services Used by lowest monthly charges customer: (1,)

Average Monthly Charges: (Decimal('64.7982081911262799'),)

Customers with less than or equal to average, and more than average monthly charges:
('<$65 (Average)', 3130)
('>$65 (More than Average)', 3902)
Monthly Charges statistics(Average, Max, Min, Standard Deviation): (Decimal('64.7982081911262799'), Decimal('118.75'), Decimal('18.25'), Decimal('30.0859738840498435'))


#### - Querying on Total Charges

- The **highest total charges** are **$8684.8**, of 1 customer, who's availing all the services, pays a high monthly charge of **$117.8** has a tenure of **72 months**, a **1 year** contract, and is predicted to churn
- The **lowest total charges** are **$18.8**, of 1 customer, who's only availing one service (Phone Service), has a low monthly charge of **$18.8**, has a tenure of **1 month**, a **1 year** contract, and is predicted not to churn
- The **average total charges** are **$2283.3** ($2300) rounded up
- **4397 customers** are being billed **less than or equal to the average**, while the remaining **2635 customers** are paying **more than the average**

In [12]:
print("Highest Total Charges:", end=' '); show(''' select max(total_charges) from billing; ''')
# print("Customers with highest total charges:", end=' '); show(''' select count(*) from billing where total_charges = (select max(total_charges) from billing); ''')
print("Services availed by customer with highest total charges:", end=' '); show(''' select sum(
    (case when phone_service then 1 else 0 end) +
    (case when multiple_lines then 1 else 0 end) +
    (case when internet_service is not null then 1 else 0 end) +
    (case when online_security then 1 else 0 end) +
    (case when online_backup then 1 else 0 end) +
    (case when device_protection then 1 else 0 end) +
    (case when tech_support then 1 else 0 end) +
    (case when streaming_tv then 1 else 0 end) +
    (case when streaming_movies then 1 else 0 end)
) as total_services
from public.services
where customer_id in (
    select customer_id
    from public.billing
    where total_charges = (select max(total_charges) from public.billing)
); ''')
print("Info about customer paying highest total charges and their prediction", end=' '); show('''SELECT 
    c.tenure, 
    b.monthly_charges, 
    b.contract, 
    b.paperless_billing, 
    b.payment_method, 
    cp.churn_prediction
FROM 
    billing b
JOIN 
    customers c ON b.customer_id = c.customer_id
JOIN 
    churn_predictions cp ON b.customer_id = cp.customer_id
WHERE 
    b.total_charges = (SELECT MAX(total_charges) FROM billing);
''')

print("\nLowest Total Charges:", end=' '); show(''' select min(total_charges) from billing; ''')
# print("Customers with lowest total charges:", end=' '); show(''' select count(*) from billing where total_charges = (select min(total_charges) from billing); ''')
print("Services availed by customer with lowest total charges:", end=' '); show(''' select sum(
    (case when phone_service then 1 else 0 end) +
    (case when multiple_lines then 1 else 0 end) +
	(case when internet_service != 'False' then 1 else 0 end) +
    (case when online_security then 1 else 0 end) +
    (case when online_backup then 1 else 0 end) +
    (case when device_protection then 1 else 0 end) +
    (case when tech_support then 1 else 0 end) +
    (case when streaming_tv then 1 else 0 end) +
    (case when streaming_movies then 1 else 0 end)
) as total_services
from public.services
where customer_id in (
    select customer_id
    from public.billing
    where total_charges = (select min(total_charges) from public.billing)
); ''')
print("Info of customer paying lowest total charges and their prediction", end=' '); show('''SELECT 
    c.tenure, 
    b.monthly_charges, 
    b.contract, 
    b.paperless_billing, 
    b.payment_method, 
    cp.churn_prediction
FROM 
    billing b
JOIN 
    customers c ON b.customer_id = c.customer_id
JOIN 
    churn_predictions cp ON b.customer_id = cp.customer_id
WHERE 
    b.total_charges = (SELECT MIN(total_charges) FROM billing);''')

print("\nAverage Total Charges:", end=' '); show(''' select avg(total_charges) from billing; ''')
print("\nCustomers with less than or equal to average, and more than average total charges:"); show(''' select 
	case
		when total_charges < 2300 then '<=$2300 (Less then equal to Average)'
		else '>$2300 (More than Average)'
	end as tcr,
	count(*) as NumberOfCustomers
from billing
group by tcr
order by tcr; ''')

print("Total Charges statistics(Average, Max, Min, Standard Deviation):", end=' '); show(''' select
	avg(total_charges) as AverageTotalCharges,
	max(total_charges) as HighestTotalCharges,
	min(total_charges) as LowestTotalCharges,
	stddev(total_charges) as StddevTotalCharges
from billing; ''')

Highest Total Charges: (Decimal('8684.8'),)
Services availed by customer with highest total charges: (9,)
Info about customer paying highest total charges and their prediction (72, Decimal('117.8'), 'One year', True, 'Bank transfer (automatic)', True)

Lowest Total Charges: (Decimal('18.8'),)
Services availed by customer with lowest total charges: (1,)
Info of customer paying lowest total charges and their prediction (1, Decimal('18.8'), 'One year', False, 'Mailed check', False)

Average Total Charges: (Decimal('2283.3004408418657565'),)

Customers with less than or equal to average, and more than average total charges:
('<=$2300 (Less then equal to Average)', 4397)
('>$2300 (More than Average)', 2635)
Total Charges statistics(Average, Max, Min, Standard Deviation): (Decimal('2283.3004408418657565'), Decimal('8684.8'), Decimal('18.8'), Decimal('2266.771361883145'))


#### - Querying on Contracts

| Contract Type | Number of Customers | Average Monthly Charges | Average Total Charges | Churned Customers | Churn Rate |
| --- | --- | --- | --- | --- | --- | 
| Month-to-Month | 3875 | 66.4 | 1369.25 | 1763 | 45.49 |
| One Year | 1472 | 65.07 | 3034.68 | 83 | 5.6 |
| Two Year | 1685 | 60.87 | 3728.93 | 4 | 0.24 |

- **Month-to-Month** contracts have the **highest churn rate** of **45.49%** with **1763 customers** churning out of **3875**, they also have the highest number of customers
- **Month-to-Month** contracts also have the **highest average monthly charges** of **$66.4** and the **lowest average total charges** of **$1369.25** (which might be due to the high churn rate, hence their total charges don't cumulate much)
- **Two Year** contracts have the **lowest churn rate** of **0.24%** with **4 customers** churning out of **1685**, while having the **lowest average monthly charges** of **$60.87**, yet still having a considerable amount of customers

In [13]:
print("Contract Type in relation to total customers, average monthly charges, average total charges, and churn predictions and rates:")
cursor.execute(''' select b.contract, count(c.customer_id) as TotalCustomers, avg(monthly_charges) as AverageMonthlyCharges, avg(total_charges) as AverageTotalCharges,
       sum(case when cp.churn_prediction = true then 1 else 0 end) as ChurnedCustomers,
       (sum(case when cp.churn_prediction = true then 1 else 0 end) * 100.0 / count(c.customer_id)) as ChurnRate
from customers c
join billing b on c.customer_id = b.customer_id
join churn_predictions cp on c.customer_id = cp.customer_id
group by b.contract
order by b.contract; ''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
contract_summary = pd.DataFrame(rows, columns=cols)
contract_summary

Contract Type in relation to total customers, average monthly charges, average total charges, and churn predictions and rates:


,contract,totalcustomers,averagemonthlycharges,averagetotalcharges,churnedcustomers,churnrate
0,Month-to-month,3875,66.3984903225806452,1369.2545806451612903,1763,45.4967741935483871
1,One year,1472,65.0794157608695652,3034.6830842391304348,83,5.6385869565217391
2,Two year,1685,60.8723738872403561,3728.9339465875370920,4,0.23738872403560830861


#### - Querying on Payment Methods

| Payment Method | Number of Customers | Average Monthly Charges | Average Total Charges | Churned Customers | Churn Rate |
| --- | --- | --- | --- | --- | --- |
| Bank Transfer (Automatic) | 1542 | 67.21 | 3079.30 | 213 | 13.81 |
| Credit Card (Automatic) | 1521 | 66.52 | 3071.40 | 166 | 10.91 |
| Electronic Check | 2365 | 76.26 | 2090.90 | 1152 | 48.71 |
| Mailed Check | 1604 | 43.96 | 1054.48 | 319 | 19.89 |

- **Electronic Check** payment method has the **highest churn rate** of **48.71%** with **1152 customers** churning out of **2365**, they also have the **highest average monthly charges** of **$76.26** and the **lowest average total charges** of **$2090.90** (again which might be due to high churn rate)
- **Credit Card (Automatic)** payment method has the **lowest churn rate** of **10.91%** with **166 customers** churning out of **1521**, they also have the **second highest average monthly charges** of **$66.52** and the **second highest average total charges** of **$3071.40**, but also have the least number of customers

In [14]:
print("Payment method in relation to total customers, average monthly charges, average total charges, and churn predictions and rates:")
cursor.execute(''' select b.payment_method, count(c.customer_id) as TotalCustomers, avg(b.monthly_charges) as AverageMonthlyCharges, avg(b.total_charges) as AverageTotalCharges,
	sum(case when cp.churn_prediction = true then 1 else 0 end) as ChurnedCustomers,
	(sum(case when cp.churn_prediction = true then 1 else 0 end) * 100.0 / count(c.customer_id)) as ChurnRate
from customers as c
join billing b on c.customer_id = b.customer_id
join churn_predictions cp on c.customer_id = cp.customer_id
group by b.payment_method
order by b.payment_method;
 ''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
payment_method_summary = pd.DataFrame(rows, columns=cols)
payment_method_summary

Payment method in relation to total customers, average monthly charges, average total charges, and churn predictions and rates:


,payment_method,totalcustomers,averagemonthlycharges,averagetotalcharges,churnedcustomers,churnrate
0,Bank transfer (automatic),1542,67.2055771725032425,3079.2995460440985733,213,13.8132295719844358
1,Credit card (automatic),1521,66.5192636423405654,3071.3960223537146614,166,10.9138724523339908
2,Electronic check,2365,76.2558139534883721,2090.8681818181818182,1152,48.7103594080338266
3,Mailed check,1604,43.9583541147132170,1054.4839152119700748,319,19.8877805486284289


#### - Querying on Internet Services Being Used

| Internet Service | Number of Customers | Average Monthly Charges | Average Total Charges | Churned Customers | Churn Rate |
| --- | --- | --- | --- | --- | --- |
| DSL | 2416 | 58.09 | 2119.79 | 370 | 15.31 |
| No Service | 1520 | 21.08 | 665.22 | 101 | 6.64 |
| Fiber Optic | 3096 | 91.5 | 3205.30 | 1379 | 44.54 |

- **Fiber Optic** internet service has the **highest churn rate** of **44.54%** with **1379 customers** churning out of **3096**, they also have the **highest average monthly charges** of **$91.5** and the **highest average total charges** of **$3205.30** which may suggest that the fiber optic service might not be upto the mark and the customers are not satisfied with it
- Customers without any internet service have the **lowest churn rate** of **6.64%** with **101 customers** churning out of **1520**, they also have the **lowest average monthly charges** of **$21.08** and the **lowest average total charges** of **$665.22**

In [12]:
print("Internet Service in relation to total customers, average monthly charges, average total charges, and churn predictions and rates:")
cursor.execute(''' select s.internet_service, count(s.customer_id) as TotalCustomers, avg(b.monthly_charges) as AverageMonthlyCharges, avg(b.total_charges) as AverageTotalCharges, 
	count(case when cp.churn_prediction = true then 1 end) as ChurnPredictions,
	(count(case when cp.churn_prediction = true then 1 end) *100.0 / count(*)) as ChurnRate
from services s
join billing b on s.customer_id = b.customer_id
join churn_predictions cp on s.customer_id = cp.customer_id
group by s.internet_service
order by s.internet_service; ''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
internet_service_summary = pd.DataFrame(rows, columns=cols)
internet_service_summary

Internet Service in relation to total customers, average monthly charges, average total charges, and churn predictions and rates:


,internet_service,totalcustomers,averagemonthlycharges,averagetotalcharges,churnpredictions,churnrate
0,DSL,2416,58.0880173841059603,2119.7892591059602649,370,15.3145695364238411
1,False,1520,21.0762828947368421,665.2203289473684211,101,6.6447368421052632
2,Fiber optic,3096,91.5001291989664083,3205.3045704134366925,1379,44.5413436692506460


#### Other General Stuff

- The **total revenue** generated was **$16056168.70**
- The **total revenue** generated by predicted churned customers was **2862926.90**
- Predicted Churned Averages:

| Churn Prediction | Number of Customers | Average Monthly Charges | Average Total Charges |
| --- | --- | --- | --- |
| False | 5182 | 61.13 | 2649.66 | 
| True | 1850 | 75.07 | 1257.10 |

In [15]:
print("Total Revenue:", end=' '); show(''' select sum(total_charges) from billing; ''')
print("Total Revenue generated by predicted churned customers:", end=' '); show(''' select sum(b.total_charges) as TotalRevenueFromChurned from billing b
join churn_predictions cp on b.customer_id = cp.customer_id where cp.churn = true;''')
print("Average monthly and total charges of predicted churned customers"); cursor.execute(''' select cp.churn_prediction, count(cp.churn_prediction) as TotalCustomers, avg(b.monthly_charges) as AverageMonthlyCharges, avg(b.total_charges) as AverageTotalCharges from billing b
join churn_predictions cp on b.customer_id = cp.customer_id
group by cp.churn_prediction
order by cp.churn_prediction;''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
p_churned_avgs = pd.DataFrame(rows, columns=cols)
print(p_churned_avgs)

Total Revenue: (Decimal('16056168.70'),)
Total Revenue generated by predicted churned customers: (Decimal('2862926.90'),)
Average monthly and total charges of predicted churned customers
   churn_prediction  totalcustomers averagemonthlycharges  \
0             False            5182   61.1306831339251254   
1              True            1850   75.0712432432432432   

     averagetotalcharges  
0  2649.6579120030876110  
1  1257.1034594594594595  


#### Advanced SQL Analysis, Trends and Running Totals / Averages

##### Churn Patterns by Tenure
We see from the below table that the people with the shortest tenures are most likely to churn (predicted). It is interesting that the first 5 months have the highest churn rates in that order

In [16]:
cursor.execute(''' select c.tenure, count(cp.customer_id) as ChurnCount from customers c
join churn_predictions cp on c.customer_id = cp.customer_id
where cp.churn_prediction = true
group by c.tenure order by ChurnCount desc limit 5; ''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
churn_patterns = pd.DataFrame(rows, columns=cols)
churn_patterns

,tenure,churncount
0,1,463
1,2,139
2,3,98
3,4,90
4,5,63


##### Running Total and Moving Average on Monthly Charges

![running total monthly](running_total_monthly.png)

![moving average monthly](moving_average_monthly.png)

In [15]:
cursor.execute(''' select b.customer_id, b.monthly_charges, sum(b.monthly_charges) over (order by b.customer_id rows between unbounded preceding and current row) as RunningTotalMonthly
from billing b order by b.customer_id;''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
running_total = pd.DataFrame(rows, columns=cols)
running_total

,customer_id,monthly_charges,runningtotalmonthly
0,0002-ORFBO,65.6,65.6
1,0003-MKNFE,59.9,125.5
2,0004-TLHLJ,73.9,199.4
3,0011-IGKFF,98.0,297.4
4,0013-EXCHZ,83.9,381.3
...,...,...,...
7027,9987-LUTYD,55.15,455398.75
7028,9992-RRAMN,85.1,455483.85
7029,9992-UJOEL,50.3,455534.15
7030,9993-LHIEB,67.85,455602.00


In [16]:
cursor.execute(''' select b.customer_id, b.monthly_charges, avg(b.monthly_charges) over (order by b.customer_id rows between unbounded preceding and current row) as MovingAverageMonthly
from billing b order by b.customer_id; ''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
moving_average = pd.DataFrame(rows, columns=cols)
moving_average

,customer_id,monthly_charges,movingaveragemonthly
0,0002-ORFBO,65.6,65.6000000000000000
1,0003-MKNFE,59.9,62.7500000000000000
2,0004-TLHLJ,73.9,66.4666666666666667
3,0011-IGKFF,98.0,74.3500000000000000
4,0013-EXCHZ,83.9,76.2600000000000000
...,...,...,...
7027,9987-LUTYD,55.15,64.7977731929425157
7028,9992-RRAMN,85.1,64.8006615450277422
7029,9992-UJOEL,50.3,64.7985988620199147
7030,9993-LHIEB,67.85,64.7990328545014934


##### Running Total and Moving Average on Total Charges

![running total](running_total.png)

![moving average](moving_average_total.png)

In [17]:
cursor.execute(''' select b.customer_id, b.total_charges, sum(b.total_charges) over (order by b.customer_id rows between unbounded preceding and current row) as RunningTotal
from billing b order by b.customer_id; ''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
running_totals = pd.DataFrame(rows, columns=cols)
running_totals

,customer_id,total_charges,runningtotal
0,0002-ORFBO,593.3,593.3
1,0003-MKNFE,542.4,1135.7
2,0004-TLHLJ,280.85,1416.55
3,0011-IGKFF,1237.85,2654.40
4,0013-EXCHZ,267.4,2921.80
...,...,...,...
7027,9987-LUTYD,742.9,16045867.00
7028,9992-RRAMN,1873.7,16047740.70
7029,9992-UJOEL,92.75,16047833.45
7030,9993-LHIEB,4627.65,16052461.10


In [18]:
cursor.execute(''' select b.customer_id, b.total_charges, avg(b.total_charges) over (order by b.customer_id rows between unbounded preceding and current row) as MovingAverageTotal
from billing b order by b.customer_id; ''')
rows = cursor.fetchall(); cols = [desc[0] for desc in cursor.description]
moving_averages = pd.DataFrame(rows, columns=cols)
moving_averages

,customer_id,total_charges,movingaveragetotal
0,0002-ORFBO,593.3,593.3000000000000000
1,0003-MKNFE,542.4,567.8500000000000000
2,0004-TLHLJ,280.85,472.1833333333333333
3,0011-IGKFF,1237.85,663.6000000000000000
4,0013-EXCHZ,267.4,584.3600000000000000
...,...,...,...
7027,9987-LUTYD,742.9,2283.1341775754126352
7028,9992-RRAMN,1873.7,2283.0759282970550576
7029,9992-UJOEL,92.75,2282.7643598862019915
7030,9993-LHIEB,4627.65,2283.0978665908121178


In [19]:
cursor.close()
conn.close()